In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [28]:
data = pd.read_csv('train_NIR5Yl1(1).csv')
tes_d = pd.read_csv('test_8i3B3FC(1).csv')

In [53]:
def stats(u):
    print(u.mean(),u.std(),u.var(),u.min(),u.max())

In [26]:
ct=100

In [4]:
data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [5]:
data.describe()

,ID,Reputation,Answers,Username,Views,Upvotes
count,330045.000000,3.300450e+05,330045.000000,330045.000000,3.300450e+05,330045.000000
mean,235748.682789,7.773147e+03,3.917672,81442.888803,2.964507e+04,337.505358
std,136039.418471,2.706141e+04,3.579515,49215.100730,8.095646e+04,3592.441135
min,1.000000,0.000000e+00,0.000000,0.000000,9.000000e+00,0.000000
25%,117909.000000,2.820000e+02,2.000000,39808.000000,2.594000e+03,8.000000
50%,235699.000000,1.236000e+03,3.000000,79010.000000,8.954000e+03,28.000000
75%,353620.000000,5.118000e+03,5.000000,122559.000000,2.687000e+04,107.000000
max,471493.000000,1.042428e+06,76.000000,175738.000000,5.231058e+06,615278.000000


In [6]:
data.corr()

tesd = tes_d.drop(columns = ['ID', 'Username'])

In [7]:
dat = data.drop(columns = ['ID', 'Username'])

In [8]:
val = dat.values
X_t = tesd.values
print(val)

[['a' 3942.0 2.0 7855.0 42.0]
 ['a' 26046.0 12.0 55801.0 1175.0]
 ['c' 1358.0 4.0 8067.0 60.0]
 ...
 ['c' 6178.0 2.0 2453.0 15.0]
 ['j' 89.0 2.0 2107.0 3.0]
 ['j' 2001.0 4.0 2554.0 37.0]]


In [9]:
X = val[:,[0,1,2,3]]
y = val[:,4]
X_t = X_t[:,[0,1,2,3]]

In [10]:
X_t.shape,X.shape

((141448, 4), (330045, 4))

In [11]:
def poly(x,k):
    temp = np.array(x)
    for i in range(0,k):
        if i ==1:
            continue
        temp = np.concatenate((temp,np.power(x,i)), axis = 1)
    return temp

In [12]:
m = {}
s = set([i for i in data["Tag"]])
tag_wise = {}
for i in s:
    temp = [data['Upvotes'][j] for j in range(len(data['Tag'])) if data['Tag'][j] == i]
    tag_wise[i] = temp
    m[i] = sum(temp)/len(temp)

In [13]:
for i in range(len(X)):
    X[i][0] = m[X[i][0]]

In [14]:
for i in range(len(X_t)):
    X_t[i][0] = m[X_t[i][0]]

In [15]:
st = {}
for i in s:
    st[i] = np.asarray(tag_wise[i]).std()

In [16]:
temp = np.array([st[i] for i in data["Tag"]])
t = np.array([np.concatenate((X[i],np.array([temp[i]]))) for i in range(len(temp))])
X = t

In [17]:
temp = np.array([st[i] for i in tesd["Tag"]])
t = np.array([np.concatenate((X_t[i],np.array([temp[i]]))) for i in range(len(temp))])
X_t = t

In [18]:
X_t.shape,X.shape,y.shape

((141448, 5), (330045, 5), (330045,))

In [57]:
X_train = poly(X,11)

In [58]:
X_test = poly(X_t,11)

In [59]:
X_train.shape,X_test.shape

((330045, 55), (141448, 55))

In [60]:
X_train[0],X_test[0]

(array([316.4601356680864, 3942.0, 2.0, 7855.0, 3106.245093052999, 1.0,
        1.0, 1.0, 1.0, 1.0, 100147.01746706366, 15539364.0, 4.0,
        61701025.0, 9648758.578115834, 31692538.734381188, 61256172888.0,
        8.0, 484661551375.0, 29971408987.32534, 10029425107.548355,
        241471833524496.0, 16.0, 3807016486050625.0, 93098542098763.89,
        3173913230207.6646, 9.518819677535633e+17, 32.0,
        2.990411449792766e+19, 2.891868895646734e+17, 1004417011430252.0,
        3.7523187168845465e+21, 64.0, 2.3489681938122178e+23,
        8.982853566855263e+20, 3.178579437045514e+17,
        1.4791640381958882e+25, 128.0, 1.845114516239497e+27,
        2.790294481365779e+24, 1.0058936798792132e+20,
        5.830864638568191e+28, 256.0, 1.4493374525061248e+31,
        8.667338540915313e+27, 3.183252504022465e+22,
        2.298526840523581e+32, 512.0, 1.138454568943561e+35,
        2.6922877812547334e+31, 1.0073725192887253e+25,
        9.060792805343956e+35, 1024.0, 8.94256063905

In [61]:
sca = StandardScaler().fit(X_train)

/Users/kk3799/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [62]:
X_tr = sca.transform(X_train)
X_te = sca.transform(X_test)

/Users/kk3799/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [63]:
reg = RandomForestRegressor()
reg.fit(X_tr,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [64]:
cross_val_score(reg,X_tr,y)

array([0.87072109, 0.88068535, 0.7713854 ])

In [65]:
ans = reg.predict(X_te)

In [66]:
stats(ans),stats(y)


347.1986729482953 3393.879328853843 11518416.898821412 0.0 422410.0
337.5053583602236 3592.4356928045213 12905594.2069359 0.0 615278.0


(None, None)

In [67]:
out = pd.DataFrame({'ID':tes_d['ID'],'Upvotes':ans})
out.to_csv("sub"+str(ct)+".csv",index = 0)
ct+=1